**TRABAJO FINAL DEL CURSO DE CAPACITACIÓN EN FUNDAMENTOS DE PYTHON PARA CCSS Y GESTIÓN PÚBLICA 2022-01**

AURELIO VALENCIA

**1. Objetivos** 

El presente reporte tiene como objetivo presentar la distribución de casos de violencia escolar reportados en las instituciones educativas del Perú, según el tipo de violencia registrado y la zona geográfica en la que se encuentra la institución educativa. De esta forma, se busca explorar si existe alguna relación entre la zona geográfica, el número de reportes y el tipo de violencia reportado.

**2. Fuentes de información:**

En el presente reporte se trabaja con dos bases de datos: I. Casos de violencia escolar reportados en el Síseve, que es una plataforma web creada en el 2013 en donde cualquier persona de la comunidad educativa puede reportar casos de violencia escolar; y la II. Padrón de Instituciones Educativas, ambas bases de datos son producidas por el Ministerio de Educación. La primera se puede obtener de http://www.siseve.pe/Web/, y la segunda de http://escale.minedu.gob.pe/

In [8]:
 from google.colab import files
uploaded = files.upload()

Saving Base_final_siseve.xlsx to Base_final_siseve.xlsx
Saving Padron_escale.xlsx to Padron_escale.xlsx


In [9]:
import pandas as pd

siseve = pd.read_excel ("Base_final_siseve.xlsx", sheet_name='Sheet1')
padron = pd.read_excel ("Padron_escale.xlsx")

**Selección de las variables de la base de datos del SíseVe que nos interesan para el presente reporte:**

*   Código modular: es la variable identificadora de cada institución educativa y que va a permitir hacer el merge con el Padrón de Instituciones Educativas
*   Año: el año en el que se reportó el caso en el sistema
*   Tipo de violencia: existen tres tipos de violencia que pueden ser reportados: violencia física, psicológica y sexual. 





In [10]:
lista_siseve = ["COD_MOD", "ANIO", "TIPO_VIOLENCIA_DSC"]
siseve_small = siseve[lista_siseve].copy().reset_index()
siseve_small

,index,COD_MOD,ANIO,TIPO_VIOLENCIA_DSC
0,0,1268648,2021,Psicológica
1,1,1268648,2021,Psicológica
2,2,1645274,2021,Sexual
3,3,782664,2021,Sexual
4,4,603647,2021,Sexual
...,...,...,...,...
40635,40635,305805,2013,Física
40636,40636,662486,2013,Física
40637,40637,762914,2013,Psicológica
40638,40638,1068725,2013,Física


**Selección de las variables de la base de datos del Padrón de Instituciones Educativas que nos interesan para el presente reporte:**

*   Código modular: es la variable identificadora de cada institución educativa y que va a permitir hacer el merge con el Padrón de Instituciones Educativas
*   Zona Geográfica: nos indica en qué zona geográfica (urbano/rural) se encuentra la institución educativa.
*   Región: la región en donde se ubica la institución educativa
*   Total alumnos: número total de alumnos de la institución educativa






In [11]:
lista_padron = ["COD_MOD", "DAREACENSO", "D_DPTO", "TALUMNO"]
padron_small = padron[lista_padron].copy().reset_index()
padron_small

,index,COD_MOD,DAREACENSO,D_DPTO,TALUMNO
0,0,415547,Urbana,ANCASH,464
1,1,415638,Urbana,ANCASH,478
2,2,415646,Urbana,ANCASH,546
3,3,415877,Urbana,ANCASH,234
4,4,567206,Rural,ANCASH,67
...,...,...,...,...,...
173142,173142,3983247,Rural,JUNIN,0
173143,173143,3983241,Urbana,APURIMAC,0
173144,173144,3983242,Rural,CUSCO,0
173145,173145,3983243,Rural,CUSCO,0


**Cruce de bases de datos**

En esta sección se hace el merge de las bases de datos, se toma como referencia el código modular, la variable que más nos interesa es la de la zona geográfica y los tipos de violencia. 

In [12]:
resultado_merge = pd.merge(siseve_small,
                     padron_small,
                    left_on = ["COD_MOD"],
                    right_on = ["COD_MOD"],
                    how = 'inner', indicator = True)
resultado_merge

,index_x,COD_MOD,ANIO,TIPO_VIOLENCIA_DSC,index_y,DAREACENSO,D_DPTO,TALUMNO,_merge
0,0,1268648,2021,Psicológica,20879,Urbana,ICA,530,both
1,1,1268648,2021,Psicológica,20879,Urbana,ICA,530,both
2,479,1268648,2021,Psicológica,20879,Urbana,ICA,530,both
3,12536,1268648,2019,Física,20879,Urbana,ICA,530,both
4,35479,1268648,2015,Psicológica,20879,Urbana,ICA,530,both
...,...,...,...,...,...,...,...,...,...
40986,40612,632653,2013,Psicológica,59761,Urbana,LIMA,0,both
40987,40614,1308725,2013,Física,81523,Urbana,LIMA,524,both
40988,40620,324632,2013,Física,69423,Urbana,LIMA,487,both
40989,40631,1421734,2013,Psicológica,20953,Urbana,ICA,23,both


**Modificación de la base**

En esta sección se hace un pequeño agregado de los tipos de violencia reportada y la zona geográfica; y se hace un conteo del número de casos reportados para cada uno de los 6 grupos. 

In [16]:
nueva_agregacion = resultado_merge.groupby(['TIPO_VIOLENCIA_DSC', 'DAREACENSO']).agg({'TIPO_VIOLENCIA_DSC': ['count']})
nueva_agregacion.columns = ['count']
nueva_agregacion = nueva_agregacion.reset_index()
print(nueva_agregacion)


  TIPO_VIOLENCIA_DSC DAREACENSO  count
0             Física      Rural   3227
1             Física     Urbana  17137
2        Psicológica      Rural   2084
3        Psicológica     Urbana  11837
4             Sexual      Rural   2147
5             Sexual     Urbana   4559


**Gráfico de barras**

A continuación, se presenta el gráfico de barras que agrupa a los tipos de violencia con la zona geográfica a la que pertenece la institución educativa en la que se reportó el caso de violencia escolar. 

In [18]:
import altair as alt

In [45]:
from altair.vegalite.v4.schema.core import Text
alt.Chart(nueva_agregacion, height = 500, width = 500).mark_bar().encode(
    column="DAREACENSO",
    x = alt.X("TIPO_VIOLENCIA_DSC", title = "Tipos de violencia escolar"),
    y = alt.Y("count", title = "Número de casos reportados", scale = alt.Scale()),
    color = alt.Color("DAREACENSO"),
    ).interactive().properties(
    title={
      "text": ["Casos reportados en el SíseVe según el tipo de violencia escolar y la zona geográfica"], 
      "subtitle": ["Fuente: Síseve 2021, Ministerio de Educación | Padrón de instituciones educativas 2022, Ministerio de Educación"],
      "color": "Black",
      "subtitleColor": "Gray"
    })

alt.Chart(...)

**Interpretación**

El gráfico "Casos reportados en el SíseVe según el tipo de violencia escolar y la zona geográfica" muestra la distribución del número de casos reportados según el tipo de violencia escolar y según la zona geográfica, esta distribución nos puede dar las siguientes ideas exploratorias: 

*   El número de casos de violencia escolar reportados en la zona urbana es mucho mayor al número de casos reportados en la zona rural. El 82% de los casos reportados pertecen a la zona urbana. ¿Existe mayor violencia en la zona urbana? Es una premisa que no podemos afirmar. Sin embargo, plantearía que no, que la razón por la que hay mayores casos reportados en la zona urbana es porque existe mayor conectividad a Internet, que es por donde se reportan la mayoría de casos de violencia escolar. 

*   La distribución de casos según el tipo de violencia en cada zona geográfica es bastante diferenciada. En la zona rural se observa que los tipos de violencia tienen un número similar de casos reportados. Entones, por ejemplo, se podría plantear que la violencia sexual es mayor en la zona rural que en la zona urbana. En la zona rural, la violencia sexual representa el 29% del total de casos reportados, mientras que en la zona urbana representa el 14%. De igual forma, que la violencia física es mayor en la zona urbana, ya que representa el 51% del total de casos, mientras que en la zona rural el 43%.

*   Para futuros análisis podría observarse el porcentaje de casos de violencia escolar según el tipo de violencia y las regiones del país.


